In [ ]:
import pandas as pd
import GladUtil as gd
import os
import duckdb
path ='e:/글로벌백업(180329)/04.수수료작업/'
years = ['2021','2022','2023','2024']
ym_list = [ i+str(j+1).rjust(2,'0') for i in years for j in range(12)]
#작업대상 기간 
ym_list = [ i for i in ym_list if i >= '202104' and i <'202404']

#파일목록 추출
file_list = []
for i in ym_list:
    path_name = f'{path}{i}수수료/'
    f_list = os.listdir(path_name)
    file_list = file_list + [ path_name + file for file in f_list if file.startswith("마감실적리스트") and
                 ('장기' in file or '생보' in file or '생명' in file)]
#미유지건 수수료 명세    
df_comm = pd.DataFrame()
cols = ['지점','담당','보험회사','정산월구분','월납보험료','수수료S','구분','증권번호','영수일','납입회차']
for full_name in file_list:
    # print(full_name)
    jong = '장기' if '장기' in full_name else '생보'
    df2 = pd.read_excel(full_name,header=0,usecols=cols, dtype={'증권번호':object,'정산월구분':object}) 
    df2 = df2[~df2['구분'].str.contains('초회|계속')]
    df2['보종'] = jong
    df_comm = pd.concat([df_comm,df2])
print('end of file list')

df_comm['정산월구분'] = df_comm['정산월구분'].str.replace('-','')

In [179]:
# 증권번호 기준으로 제일 늦게 환수가 발생한 건만 남김
df_temp = df_comm[df_comm['수수료S'] <= 0].sort_values(['증권번호','영수일'])
df_comm2 = df_temp.drop_duplicates(['증권번호'], keep='last')  #증번중복은 마지막 건만 추출

In [130]:
'''금김원 제출대상 목록'''
path ='e:/글로벌백업(180329)/금감원/GLAD.xlsx'
df_list = pd.read_excel(path,header=0,dtype={'증권번호':object}) 

C:\Users\admin\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [186]:
# join 
query_string = f'''
     SELECT a.*, b.보험회사,left(b.정산월구분,6) as 정산월, b.월납보험료, b.수수료S, b.구분, b.영수일, b.보종
          from df_list a LEFT OUTER JOIN df_comm2 b
            ON a.증권번호 = b.증권번호 '''
df_result = duckdb.query(query_string).to_df()
df_result.to_clipboard()

## 이하는 확인용

In [197]:
pl_no = '42145013090000'
df_comm[df_comm['증권번호']==pl_no]
# df_temp[df_temp.증권번호==pl_no]
# df_result[df_result.증권번호==pl_no]
# df_comm2[df_comm2.증권번호==pl_no]

,지점,담당,보험회사,정산월구분,월납보험료,수수료S,구분,납입회차,영수일,증권번호,보종
1728,글래드아이리치,이미경,흥국화재,NaN,-59999,-20748,실효,1,2021-10-01,42145013090000,장기


In [196]:
df_comm[df_comm['정산월구분']=='NaN']

,지점,담당,보험회사,정산월구분,월납보험료,수수료S,구분,납입회차,영수일,증권번호,보종
